# SQL demo

This project is somewhat different from my other projects, in that its goal is simply to give the reader an idea of my SQL skills. It's not a fully-fledged project where I'm analysing something specific; rather, it's a series of examples of use cases of SQL for data manipulation and data defintion.

## SQL as DML

In this section, I'm going to query the `chinook` demo database to visualise, add, update, or delete information. I downloaded the database from Kaggle [here](https://www.kaggle.com/datasets/samaxtech/chinook-music-store-data). For your reference, below is the database schema.

![img](chinook_schema.png)

### Vanilla SQL

Let's start this demo with SQL only. Later on, we'll combine SQL with Python to create queries more dynamically.

In [1]:
%load_ext sql
%sql sqlite:///chinook.db

#### Agent sales statistics

The query below shows a summary of each sales support agent in the database: their name, the date they were hired, the total amount of their sales, the number of their sales, and the average amount per sale. This approach makes extensive use of nested subqueries and can be hard to read, although the code is fairly short.

In [2]:
%%sql
    SELECT *, 
           ROUND(sales_total / number_of_sales, 2) AS avg_sale
    FROM (SELECT e.first_name || ' ' || e.last_name AS sales_agent,           
                 STRFTIME("%d.%m.%Y", e.hire_date) AS hire_date,
                 ROUND(SUM(sales.cust_total)) AS sales_total,
                 SUM(sales.nr_sales) AS number_of_sales        
            FROM employee e
            LEFT JOIN (SELECT c.support_rep_id, 
                              c.customer_id, 
                              SUM(i.total) AS cust_total,
                              COUNT(i.invoice_id) AS nr_sales
                        FROM customer c
                        LEFT JOIN invoice i
                          ON i.customer_id = c.customer_id
                        GROUP BY c.customer_id ) AS sales
             ON e.employee_id = sales.support_rep_id
             WHERE e.title = "Sales Support Agent"
             GROUP BY sales_agent)

 * sqlite:///chinook.db
Done.


sales_agent,hire_date,sales_total,number_of_sales,avg_sale
Jane Peacock,01.04.2017,1732.0,212,8.17
Margaret Park,03.05.2017,1584.0,214,7.4
Steve Johnson,17.10.2017,1394.0,188,7.41


Another option involves longer but better-organised code, using a bunch of `WITH` statements, which may also be reused for other purposes.

In [3]:
%%sql
    WITH invoices_per_customer AS (
        SELECT customer_id, 
               COUNT(invoice_id) AS nr_invoices,
               ROUND(SUM(total), 2) AS customer_total
        FROM invoice
        GROUP BY customer_id
    ),
    
    agent_customers AS (
        SELECT support_rep_id AS agent_id,
               customer_id
        FROM customer
        WHERE support_rep_id IN (SELECT agent_id FROM agents)
    ),
    
    agents AS (
        SELECT employee_id AS agent_id,
               first_name || " " || last_name AS sales_agent,
               STRFTIME("%d.%m.%Y", hire_date) AS hire_date
        FROM employee
        WHERE title = "Sales Support Agent"
    ),
    
    agent_sales AS (        
        SELECT a.agent_id,
               sales_agent,
               ROUND(SUM(customer_total)) AS sales_total,
               SUM(nr_invoices) AS number_of_sales
          FROM agents a
            LEFT JOIN agent_customers ac
               ON a.agent_id = ac.agent_id
                 LEFT JOIN invoices_per_customer ic
                  ON ic.customer_id = ac.customer_id    
            GROUP BY sales_agent 
    )
    
    SELECT a.sales_agent,
           a.hire_date,
           a_sales.sales_total,
           a_sales.number_of_sales,
           ROUND(sales_total / number_of_sales, 2) AS avg_sale
    FROM agents a
      LEFT JOIN agent_sales a_sales
        ON a_sales.agent_id = a.agent_id      

 * sqlite:///chinook.db
Done.


sales_agent,hire_date,sales_total,number_of_sales,avg_sale
Jane Peacock,01.04.2017,1732.0,212,8.17
Margaret Park,03.05.2017,1584.0,214,7.4
Steve Johnson,17.10.2017,1394.0,188,7.41


#### Total purchases per customer

To find out how many purchases each customer has made, it's enough to left-join `customers` with `invoice` on the condition that the foreign key `customer_id` in the `invoice` table is equal to the primary key of `customer`. Since the same customer is likely to have made multiple purchases, grouping the results by `customer_id` and using the aggregate function `COUNT` will be necessary to see the actual purchase count. Note that the left join guarantees all customers will appear, and if any of them had no purchases, the count for these customer would be zero (which is not the case with this database). I'm limiting the output to the first 5 customers for your reading convenience.

In [4]:
%%sql
    SELECT c.last_name || ", " || c.first_name AS customer,
           COUNT(i.invoice_id) AS total_purchases
    FROM customer c
    LEFT JOIN invoice i
    ON c.customer_id = i.customer_id
    GROUP BY c.customer_id
    ORDER BY customer ASC
    LIMIT 5

 * sqlite:///chinook.db
Done.


customer,total_purchases
"Almeida, Roberto",11
"Barnett, Julia",10
"Bernard, Camille",9
"Brooks, Michelle",8
"Brown, Robert",4


If we wanted to find out what customers have made no purchases, we could select all customers `EXCEPT` those whose ids don't appear on any invoice. As said, there are no such customers in this case, so the result is an empty table.

In [5]:
%%sql
SELECT last_name || ", " || first_name AS customer
  FROM customer
    
EXCEPT

SELECT last_name || ", " || first_name AS customer
  FROM customer c
    WHERE c.customer_id IN (SELECT i.customer_id FROM invoice i)

 * sqlite:///chinook.db
Done.


customer


#### Top 5 customers

Let's define "top 5 customers" as the five customers that have spent the most.The query below shows their names, amount spent, and total purchases using a rather simple left join.

In [6]:
%%sql
    SELECT c.first_name || " " || c.last_name AS customer,
           ROUND(SUM(i.total)) AS total_spent,
           COUNT(i.invoice_id) AS total_purchases
    FROM customer c
        LEFT JOIN invoice i
        ON c.customer_id = i.customer_id
    GROUP BY c.customer_id
    ORDER BY total_spent DESC
    LIMIT 5

 * sqlite:///chinook.db
Done.


customer,total_spent,total_purchases
František Wichterlová,145.0,18
Helena Holý,129.0,12
Hugh O'Reilly,115.0,13
Manoj Pareek,112.0,13
Luís Gonçalves,109.0,13


#### Artist statistics

Finding out how many albums and how many tracks there are per artists is again a matter of a simple left join. Note that I consider a track to be associated with an artist if the track is part of an album that is associated with the artist. (I say this because each track has a `composer` who might be different from the artist linked to the album.) Again, I'm limiting the results to the first 5 to avoid cluttering the page.

In [7]:
%%sql
SELECT a.name AS artist, 
       COUNT(DISTINCT(al.album_id)) AS total_albums,
       COUNT(t.track_id) AS total_tracks
FROM artist a
LEFT JOIN album al
ON a.artist_id = al.artist_id
LEFT JOIN track t
ON al.album_id = t.album_id
GROUP BY artist
ORDER BY total_albums DESC, total_tracks DESC
LIMIT 5

 * sqlite:///chinook.db
Done.


artist,total_albums,total_tracks
Iron Maiden,21,213
Led Zeppelin,14,114
Deep Purple,11,92
U2,10,135
Metallica,10,112


#### Most popular genres, by revenue and tracks sold

Say we wanted to see which music genres are best sellers, in terms of revenue and tracks sold. To do that, we need to join the `track`, `genre` and the `invoice_line` tables, so that we can calculate both variables. Note that the result table specifically includes also any genres that didn't sell at all.

In [8]:
%%sql
    SELECT g.name AS genre,
           IFNULL(genre_stats.total_revenue, 0) AS total_revenue,
           IFNULL(genre_stats.tracks_sold, 0) AS tracks_sold
    FROM genre g
    LEFT JOIN 
        (SELECT t.genre_id,
               ROUND(SUM(il.unit_price * il.quantity)) AS total_revenue,
               SUM(il.quantity) AS tracks_sold
        FROM track t
            INNER JOIN invoice_line il
            ON t.track_id = il.track_id
        GROUP BY genre_id) AS genre_stats
    ON genre_stats.genre_id = g.genre_id
    ORDER BY total_revenue DESC, tracks_sold DESC

 * sqlite:///chinook.db
Done.


genre,total_revenue,tracks_sold
Rock,2609.0,2635
Metal,613.0,619
Alternative & Punk,487.0,492
Latin,165.0,167
R&B/Soul,157.0,159
Blues,123.0,124
Jazz,120.0,121
Alternative,116.0,117
Easy Listening,73.0,74
Pop,62.0,63


#### Tracks that don't sell

Tracks that don't sell—i.e., tracks that never appear in any invoice—are tracks whose `id` is nowhere to be found in the `track_id` column of the `invoice_line` table. There are a couple of ways they can be identified.
The first way is to left-join the `track` and `invoice_line` tables on the condition that `track_id` is identical for both tables. Since it's a left join, each and every `track_id` in the `track` table will appear in the final result; however, for any track that didn't sell, there _is_ no invoice line (or invoice), so any columns inherited from the `invoice_line` table will be null. At this point, we can filter the result by asking only for those rows that have null `invoice_id`. The list of unsold tracks is quite long, so again, I'll limit to, say, the first 10.

In [9]:
%%sql
SELECT t.name AS track_name, 
       g.name AS genre       
FROM track t
    LEFT JOIN invoice_line il
    ON t.track_id = il.track_id
        LEFT JOIN genre g
        ON t.genre_id = g.genre_id
WHERE il.invoice_id IS NULL
ORDER BY genre ASC
LIMIT 10

 * sqlite:///chinook.db
Done.


track_name,genre
Wooden Jesus,Alternative
Your Savior,Alternative
Revelations,Alternative
Sound of a Gun,Alternative
Jewel of the Summertime,Alternative
"Band Members Discuss Tracks from ""Revelations""",Alternative
Your Time Has Come,Alternative & Punk
Be Yourself,Alternative & Punk
Heaven's Dead,Alternative & Punk
Man Or Animal,Alternative & Punk


Another way that is perhaps conceptually more simple is to extract all `track_id`s that never appear in `invoice_line`, and then use them to filter the result of a joined query similar to the above. This approach uses the set-theoretical operation `EXCEPT` and only one `JOIN`, unlike the previous one. The same approach could be taken if we replaced `EXCEPT` with `WHERE t.track_id NOT IN` and then the same subquery.

In [10]:
%%sql
SELECT t.name AS track_name, 
       g.name AS genre  
FROM track t
LEFT JOIN genre g
ON t.genre_id = g.genre_id
WHERE t.track_id IN (SELECT t.track_id
                       FROM track t

                     EXCEPT

                     SELECT il.track_id
                       FROM invoice_line il)
ORDER BY genre ASC
LIMIT 10

 * sqlite:///chinook.db
Done.


track_name,genre
Wooden Jesus,Alternative
Your Savior,Alternative
Revelations,Alternative
Sound of a Gun,Alternative
Jewel of the Summertime,Alternative
"Band Members Discuss Tracks from ""Revelations""",Alternative
Your Time Has Come,Alternative & Punk
Be Yourself,Alternative & Punk
Heaven's Dead,Alternative & Punk
Man Or Animal,Alternative & Punk


#### Top 5 artists in top 5 countries

Let's define "top 5 artists" as the five artists with the most tracks bought, and "top 5 countries" as the five countries with the most customers. This is a fairly complex query that is better to break down into multiple `WITH` statements that select the top 5 countries, the tracks sold per country by each artists, and finally the count of tracks sold by each artist per country. This last query needs to be sorted by country in ascending order and tracks sold in descending order, and then we can add a new column to it that simply numbers the rows of the resulting table over a partition, using the window function ROW_NUMBER. In simpler terms, the resulting table will be ordered by country alphabetically, by tracks sold from most to least, its rows will be numbered in ascending order (1, 2, 3, etc.), and the numbering will restart for each new country. This means that, for each country, the rows numbered 1 to 5 represent the five artists with most tracks sold for that country.

In [11]:
%%sql
    WITH     
        top_five_countries AS (
        SELECT country, 
               COUNT(customer_id) AS number_of_customers
        FROM customer
        GROUP BY country
        ORDER BY number_of_customers DESC
        LIMIT 5),

        tracks_by_invoice AS (
        SELECT i.billing_country,           
               t.track_id,
               ar.name AS artist
        FROM invoice i
            INNER JOIN invoice_line il
            ON il.invoice_id = i.invoice_id
                INNER JOIN track t
                ON t.track_id = il.track_id
                    INNER JOIN album a
                    ON a.album_id = t.album_id
                        INNER JOIN artist ar
                        ON ar.artist_id = a.artist_id),
        
        artists_by_tracks_and_country AS (
            SELECT artist,
               COUNT(track_id) AS tracks_sold,
               billing_country AS country               
            FROM tracks_by_invoice
            WHERE country IN (SELECT country FROM top_five_countries)
            GROUP BY artist
            ORDER BY country ASC, tracks_sold DESC
        ),
        
        ranked_artists AS (
            SELECT *,
                   ROW_NUMBER() OVER(PARTITION BY country ORDER BY tracks_sold DESC) AS rank
              FROM artists_by_tracks_and_country
        )
               
        SELECT artist,
               tracks_sold,
               country
        FROM ranked_artists
        WHERE rank <= 5;

 * sqlite:///chinook.db
Done.


artist,tracks_sold,country
AC/DC,58,Brazil
Apocalyptica,27,Brazil
Marisa Monte,24,Brazil
Black Label Society,8,Brazil
Tim Maia,4,Brazil
The Rolling Stones,69,Canada
R.E.M.,68,Canada
Amy Winehouse,66,Canada
Eric Clapton,62,Canada
Green Day,61,Canada


#### Inserting, updating, and deleting rows

No SQL demo project would be complete without manipulating data. Say we got two new employees hired today to add the database. We know all about them, except their email, because they'll have a company address that hasn't been created yet. They're both salespeople and they both report to Nancy Edwards, whose id is 2. Before inserting anything, let's check what type of data each column requires.

In [12]:
%%sql
PRAGMA table_info(employee)

 * sqlite:///chinook.db
Done.


cid,name,type,notnull,dflt_value,pk
0,employee_id,INTEGER,1,None,1
1,last_name,NVARCHAR(20),1,None,0
2,first_name,NVARCHAR(20),1,None,0
3,title,NVARCHAR(30),0,None,0
4,reports_to,INTEGER,0,None,0
5,birthdate,DATETIME,0,None,0
6,hire_date,DATETIME,0,None,0
7,address,NVARCHAR(70),0,None,0
8,city,NVARCHAR(40),0,None,0
9,state,NVARCHAR(40),0,None,0


It would make sense for `employee_id` to autoincrement with any new rom inserted, but we can't tell if it will from the table above. We can check that with this code:

In [13]:
%%sql
    SELECT sql FROM sqlite_master WHERE tbl_name = "employee"

 * sqlite:///chinook.db
Done.


sql
"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
CREATE INDEX [ifk_employeereports_to] ON [employee] ([reports_to])


The keyword `AUTOINCREMENT` doesn't appear in the code that defines the `employee_id` column, so we'll need to manually specify the next ids ourselves—they're 9 and 10, respectively.

In [14]:
%%sql
    INSERT INTO employee (employee_id, last_name, first_name, title, reports_to, birthdate, hire_date, address, city, state, country, postal_code, phone, fax)
    VALUES
        (9, "Doe", "John", "Sales Support Agent", 2, DATE("1991-05-13"), DATE("now"), "Black street, 4", "New York", "New York", "USA", "12345", "034586847", "034586846"),
        (10, "Contrary", "Mary", "Sales Support Agent", 2, DATE("1990-11-21"), DATE("now"), "Green bulevard, 8", "Old York", "Some state", "USA", "54321", "0129427422", "0129427421")

 * sqlite:///chinook.db
2 rows affected.


[]

Let's check the result.

In [15]:
%%sql
    SELECT * FROM employee WHERE employee_id > 8

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
9,Doe,John,Sales Support Agent,2,1991-05-13,2022-04-21,"Black street, 4",New York,New York,USA,12345,034586847,034586846,None
10,Contrary,Mary,Sales Support Agent,2,1990-11-21,2022-04-21,"Green bulevard, 8",Old York,Some state,USA,54321,0129427422,0129427421,None


Great. Now say that we know the emails: `john.doe@chinookcorp.com` and `mary.contrary@chinookcorp.com`. To add these two values, we use an `UPDATE` statement.

In [16]:
%%sql
    UPDATE employee
    SET email = first_name || "." || last_name || "@chinookcorp.com"    
    WHERE employee_id IN (9, 10);
    
    SELECT * FROM employee WHERE employee_id > 8

 * sqlite:///chinook.db
2 rows affected.
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
9,Doe,John,Sales Support Agent,2,1991-05-13,2022-04-21,"Black street, 4",New York,New York,USA,12345,034586847,034586846,John.Doe@chinookcorp.com
10,Contrary,Mary,Sales Support Agent,2,1990-11-21,2022-04-21,"Green bulevard, 8",Old York,Some state,USA,54321,0129427422,0129427421,Mary.Contrary@chinookcorp.com


Now, John's and Mary's contract was _veeery_ short. They're done already, and we need to remove them from the database. Before nuking _anything_ from a table, it's good practice to double-check that we're actually deleting the right stuff.

In [17]:
%%sql
    SELECT * FROM employee WHERE employee_id IN (9, 10)

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
9,Doe,John,Sales Support Agent,2,1991-05-13,2022-04-21,"Black street, 4",New York,New York,USA,12345,034586847,034586846,John.Doe@chinookcorp.com
10,Contrary,Mary,Sales Support Agent,2,1990-11-21,2022-04-21,"Green bulevard, 8",Old York,Some state,USA,54321,0129427422,0129427421,Mary.Contrary@chinookcorp.com


Now that we're sure, let's delete their records.

In [18]:
%%sql
    DELETE FROM employee WHERE employee_id IN (9, 10);
    SELECT * FROM employee

 * sqlite:///chinook.db
2 rows affected.
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


### SQL & Python

In this section, I'm going to use Python to create a few SQL queries in a more dynamic fashion. I'll use the `sqlite3` module to interrogate the database, and `pandas` to quickly display results in a convenient format.

In [19]:
import sqlite3
import pandas as pd

Note that, depending on the context, the input of the functions below would need to be sanitised first. If the functions were only accessible from a GUI where input values can only be chosen from a list, for example, this wouldn't be necessary as there would be no way to input wrong or dangerous data. If they were accessible from other code, or from a GUI where write-ins are possible, then the input should go through some kind of validation\sanitisation function first, to prevent SQL injection attacks or the insertion of senseless data. I did do some input validation, but making the functions bullet-proof is beyond the scope of this demo, and more thorough validation would probably be necessary in a production environment.

#### Total purchases per customer

The function below does pretty much the same as its vanilla SQL counterpart. Just like that one, it can only sort by one column (`total_spent` by default), but the sorting order is specifiable, as is the number of rows to show. Optionally, it's possible to visualise only the customers that made no purchases.

In [20]:
def customer_purchases(country = None, order_by = "total_spent", order = "DESC", limit = None, no_purchases = False):
    """Extracts customer purchase information from the database, and returns it in DataFrame format.
    
    `country`: customer country, None by default
    `order_by`: single string indicating a database column by which results should be sorted. Expects `customer`, `total_spent` (default), or `total_purchases`.
    `order`: string specifying sorting order, `DESC` (default) or `ASC`
    `limit`: max number of rows to show, default None
    `no_purchase`: boolean. If True, only show customers with no purchases. (Accepts truthy and falsey values.)
    
    Returns: DataFrame    
    """
    # Validate passed parameters.
    if order_by not in ["total_spent", "total_purchases"]:
        return "Error: unknown sorting column, '{}'".format(order_by)
    
    if order not in ["ASC", "DESC"]:
        return "Error: unknown sorting option, '{}'".format(order)
    
    if limit is not None and (type(limit) != int or limit < 1):
        return "Error: `limit` must be a positive integer."    
    
    # The base query, on the assumption that we want to show all customers.
    query = """SELECT c.last_name || ", " || c.first_name AS customer,
                      ROUND(SUM(i.total)) AS total_spent,
                      COUNT(i.invoice_id) AS total_purchases
                FROM customer c
                    LEFT JOIN invoice i
                    ON c.customer_id = i.customer_id\n"""
    
    # Filter by country.
    if country is not None: 
        query = query + """WHERE country = '{}'\n""".format(country)
        
    # Group by customer.
    query = query + """GROUP BY c.customer_id\n"""
    
    # Filter by customers with no purchases only, if needed.
    if no_purchases:
        query = query + """HAVING total_purchases = 0\n"""
    
    # Sort.
    query = query + """ORDER BY {} {}\n""".format(order_by, order)
    
    # Limit results.
    if limit is not None:
        query = query + "LIMIT {}".format(limit)
    
    # Connect and execute.
    conn = sqlite3.connect("chinook.db")
    cur = conn.cursor()    
    cur.execute(query)
    data = cur.fetchall()
    conn.close()
    
    # Return the data as a DataFrame.
    result = pd.DataFrame(columns = ["customer", "total_spent", "total_purchases"], data = data)    
    return result    

So, for example, let's say that I want to see the top-3 customers (in terms of money spent) from the United States. (If you're running this code, you can try other uses of the same function as well.)

In [21]:
customer_purchases(country = "USA", limit = 3)

,customer,total_spent,total_purchases
0,"Smith, Jack",98.0,12
1,"Miller, Dan",95.0,12
2,"Leacock, Heather",92.0,12


#### Genre statistics

The function below displays all genres in the database, together with the number of tracks of that genre, the total revenue from selling tracks of each genre, and the number of tracks of each genre sold. It allows for multiple sorting columns and to display only genres with no sales by passing an appropriate parameter.

In [22]:
def genre_statistics(order = [("genre", "ASC")], no_sales = False):
    """For each genre, computes the total of tracks available, total tracks sold, and total revenue, and returns these data in DataFrame format.
    
    `order`: an array of tuples specifying the columns to sort by (`num_tracks`, `total_revenue`, or `tracks_sold`) and the sorting order (`ASC` or `DESC`)
    `no_sales`: boolean. If True, show only genres with no tracks sold. (Accepts truthy and falsey values.)
    """
    
    # Validate `order`.
    if type(order) != list:
        return type(order)
        return "Error: `order` must be a list of 2-tuples."
    
    if any([(type(x) != tuple) or (len(x) != 2) for x in order]):
        return "Error: `order` contains non-tuples or tuples of length other than 2."
    
    valid_columns = ["genre", "num_tracks", "total_revenue", "tracks_sold"]
    valid_sorts = ["ASC", "DESC"]
    
    if any([(x[0] not in valid_columns) for x in order]):
        return "Error: unknown sorting column."
    
    if any([(x[1] not in valid_sorts) for x in order]):
        return "Error: unknown sort option."
           
    # The base query, on the assumption that we want to show all genres.
    query = """SELECT g.name AS genre,
                      IFNULL(genre_info.num_tracks, 0) AS num_tracks,
                      IFNULL(genre_info.total_revenue, 0.0) AS total_revenue,
                      IFNULL(genre_info.tracks_sold, 0) AS tracks_sold
               FROM genre g
                   LEFT JOIN 
                       (SELECT t.genre_id,
                          ROUND(SUM(il.unit_price * il.quantity)) AS total_revenue,
                          SUM(il.quantity) AS tracks_sold,
                          COUNT(t.track_id) AS num_tracks
                       FROM track t
                           LEFT JOIN invoice_line il
                           ON t.track_id = il.track_id
                       GROUP BY genre_id) AS genre_info
               ON genre_info.genre_id = g.genre_id\n"""
    
    # Show only tracks with no sales if necessary.
    if no_sales:
        query = query + """WHERE tracks_sold IS NULL\n""" # We check for nulls because they're the actual values that show up in the query for tracks with no sales. Zeros are added afterwards by IFNULL and checking for them won't work.
    
    # Build the sorting part of the query.
    order_query = "ORDER BY "
    for o in order:
        order_query = order_query + o[0] + " " + o[1]
        if o != order[-1]:
            order_query = order_query + ", "
    
    query = query + order_query
    
    # Connect and execute.
    conn = sqlite3.connect("chinook.db")
    cur = conn.cursor()    
    cur.execute(query)
    data = cur.fetchall()
    conn.close()
    
    # Return the data as a DataFrame.
    result = pd.DataFrame(columns = ["genre", "num_tracks", "total_revenue", "tracks_sold"], data = data) 
    return result

In this example, I'm going to show the result of calling the function with `no_sales = True`, but feel free to experiment with other parameters.

In [23]:
genre_statistics(order = [("num_tracks", "DESC")], no_sales = True)

,genre,num_tracks,total_revenue,tracks_sold
0,World,28,0.0,0
1,Sci Fi & Fantasy,26,0.0,0
2,Comedy,17,0.0,0
3,Bossa Nova,15,0.0,0
4,Science Fiction,13,0.0,0
5,Rock And Roll,12,0.0,0
6,Opera,1,0.0,0


#### Track information

This function search the database for tracks whose name contains a specific search string, and returns information about the tracks it found. Basic information (name, genre, album, artist, and composer) are always returned, but you can specify other optional information to display: size information, media information, and sales information. If you pass too many or unknown optional information parameters, it will return an error message.

In [24]:
def track_info(search_string, optional_info = []): 
    """Searches the database for `search_string` in the `name` column of the `track` table, and returns the desired information about found tracks.
    Always returns at least basic info (name, genre, album, artist, and composer), but other options are specifiable as a list assigned to the `optional_info` kwarg:
    - `size`: returns information on the track's size (in kilobytes) and duration (in seconds).
    - `media`: returns information the tracks's media format and playlists it is included in.
    - `sales`: returns information on how many tracks have been sold and what was the total revenue.
    
    Returns: dataframe, sorted by `track_name` in ascending order. If optional_info is not a list, or contains more than three options or unknown options, returns an error.
    """
    
    # Validate `optional_info`.
    if type(optional_info) != list:
        return "Error: `optional_info` must be a list of strings."
    
    if len(optional_info) > 3:
        return "Error: too many items in `optional_info` (max. 3)"
    
    allowed_options = ["size", "media", "sales"]
    for item in optional_info:
        if item not in allowed_options:
            return "Unknown option: '{}'".format(item)
    
    # Components of the query.
    track_playlists = """WITH track_playlists AS (
                           SELECT t.track_id, 
                                  GROUP_CONCAT(pl.name, ", ") AS playlists
                           FROM track t 
                             LEFT JOIN playlist_track pt
                                ON t.track_id = pt.track_id
                                  LEFT JOIN playlist pl
                                    ON pt.playlist_id = pl.playlist_id
                           GROUP BY t.track_id)\n"""    
    
    basic_select = """SELECT t.name AS track_name,
                             IFNULL(g.name, '<Unknown>') AS genre,
                             IFNULL(al.title, '<Unknown>') AS album,
                             IFNULL(a.name, '<Unknown>') AS artist,
                             IFNULL(t.composer, '<Unknown>') AS composer"""
    
    size_select = """IFNULL(t.milliseconds /1000, '<Unknown>') AS seconds,
                     IFNULL(t.bytes /1000, '<Unknown>') as kilobytes"""
    
    media_select = """IFNULL(mt.name, '<Unknown>') AS media_type,
                      IFNULL(tp.playlists, '<None>') AS playlists"""
    
    sales_select = """IFNULL(SUM(quantity), 0) AS units_sold,
                      IFNULL(ROUND(SUM(il.quantity * il.unit_price), 2), 0.0) AS total_revenue"""
        
    basic_from = """FROM track t
                      LEFT JOIN genre g
                        ON t.genre_id = g.genre_id
                      LEFT JOIN album al
                        ON t.album_id = al.album_id
                      LEFT JOIN artist a
                        ON al.artist_id = a.artist_id"""
    
    media_from = """LEFT JOIN media_type mt
                        ON t.media_type_id = mt.media_type_id
                          LEFT JOIN track_playlists tp
                            ON t.track_id = tp.track_id"""
    
    sales_from = """LEFT JOIN invoice_line il
                      ON t.track_id = il.track_id"""
    
    filter_and_group = """WHERE t.name LIKE '%{}%'
                          GROUP BY t.track_id
                          ORDER BY t.name ASC""".format(search_string)
    
    # Build the full query based on the desired options.
    full_query = ""
    columns = ["track_name", "genre", "album", "artist", "composer"] # Columns of the dataframe containing the results
    
    # Basic track information is always selected.
    select_query = basic_select    
    from_query = basic_from
    
    # Optional information.
    if "size" in optional_info:
        select_query = select_query + ",\n" + size_select
        columns = columns + ["seconds", "kilobytes"]
    if "media" in optional_info:
        full_query = track_playlists # This puts the WITH statement atop the full query.
        select_query = select_query + ",\n" + media_select
        from_query = from_query + "\n" + media_from
        columns = columns + ["media_type", "playlists"]
    if "sales" in optional_info:
        select_query = select_query + ",\n" + sales_select
        from_query = from_query + "\n" + sales_from
        columns = columns + ["units_sold", "total_revenue"]
    
    full_query = full_query + "\n" + select_query + "\n" + from_query + "\n" + filter_and_group
    
    # Connect and execute.
    conn = sqlite3.connect("chinook.db")
    cur = conn.cursor()    
    cur.execute(full_query)
    data = cur.fetchall()
    conn.close()
    
    # Return the data as a DataFrame.
    result = pd.DataFrame(columns = columns, data = data)
    return result

As an example, say we wanted to find all tracks whose name contains the word "another", and display their media and sales information. **Note:** if you see some songs appearing in the same playlist twice, that's not a mistake. For some reason, the `playlist` table contains the playlists `Music` and `TV Shows` twice, and several songs have been added to both copies of both. In a real-world application, this is _definitely_ something that should be looked into and corrected if necessary; but in this case, it's not worth wasting time on it.

In [25]:
track_info("another", optional_info = ["media", "sales"])

,track_name,genre,album,artist,composer,media_type,playlists,units_sold,total_revenue
0,Another Life,Heavy Metal,Killers,Iron Maiden,Steve Harris,MPEG audio file,"Music, Music",1,0.99
1,Another One Bites The Dust,Rock,Greatest Hits I,Queen,"Deacon, John",MPEG audio file,"Music, Music",5,4.95
2,Another Round,Rock,In Your Honor [Disc 2],Foo Fighters,"Dave Grohl, Taylor Hawkins, Nate Mendel, Chris...",MPEG audio file,"Music, Music",4,3.96
3,Jesus Of Suburbia / City Of The Damned / I Don...,Alternative & Punk,American Idiot,Green Day,Billie Joe Armstrong/Green Day,MPEG audio file,"Music, Music",4,3.96
4,Just Another Story,Electronica/Dance,The Return Of The Space Cowboy,Jamiroquai,Toby Smith,MPEG audio file,"Music, 90’s Music, Music",2,1.98
5,Raised By Another,TV Shows,"Lost, Season 1",Lost,<Unknown>,Protected MPEG-4 video file,"TV Shows, TV Shows",0,0.00
6,You've Got Another Thing Comin',Metal,Living After Midnight,Judas Priest,<Unknown>,MPEG audio file,"Music, Music",3,2.97


## SQL as DDL

In this section, I'm going to create a simple SQLite database from scratch, populate it with mock-up data, and create a few views that might be used in a real-world scenario. It's a very small library database, whose schema is displayed below. Blue cells indicate primary keys, while green ones are foreign keys.

![img](library_schema.jpg)

I was trying to keep things simple, hence there are only four tables, but other tables may include a `cards` table that displays the status of each card (e.g., active, suspended, lost...), a `loan_status` table, or a `library` table, in case this was the database of a network of libraries like Helmet in Helsinki, for example. Now, let's create the database and the tables.

In [26]:
%%sql
sqlite:///library.db

### Create the author table

In [27]:
%%sql
CREATE TABLE author(
  author_id INTEGER PRIMARY KEY AUTOINCREMENT,
  last_name TEXT NOT NULL,
  first_name TEXT NOT NULL,
  nationality TEXT NOT NULL
);

   sqlite:///chinook.db
 * sqlite:///library.db
Done.


[]

Now let's add some mock-up authors.

In [28]:
%%sql
INSERT INTO author(last_name, first_name, nationality)
VALUES
    ("Black", "Jack", "American"),
    ("Garcia", "Elena", "Spanish"),
    ("Perrier", "Milo", "French"),
    ("Fletcher", "Jessica", "British"),
    ("Conan Doyle", "Arthur", "British"),
    ("Calvino", "Italo", "Italian"),
    ("Pekkanen", "Pekka", "Finnish"),
    ("Lindstedt", "Maria", "Swedish"),
    ("Karamazova", "Irina", "Russian"),
    ("Greve", "Verena", "German")

   sqlite:///chinook.db
 * sqlite:///library.db
10 rows affected.


[]

### Create the book table

In [29]:
%%sql
CREATE TABLE book(
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author_id INTEGER NOT NULL,
    isbn CHAR(20) NOT NULL,
    language TEXT NOT NULL,
    genre TEXT NOT NULL,
    FOREIGN KEY (author_id)
        REFERENCES author (author_id)
)

   sqlite:///chinook.db
 * sqlite:///library.db
Done.


[]

Let's add books to the book table.

In [30]:
%%sql
INSERT INTO book(title, author_id, isbn, language, genre)
VALUES
    ("How to do things", 1, "978-3-16-148410-0", "American English", "Non-fiction"),
    ("How NOT to do things", 1, "948-7-25-168218-9", "American English", "Non-fiction"),
    ("SQL por tontos ", 2, "043-2-79-546711-4", "Spanish", "Programming"),
    ("Python no es una serpiente", 2, "047-8-41-295837-2", "Spanish", "Programming"),
    ("Cómo crear títulos de libros ingeniosos", 2, "738-1-88-029588-1", "Spanish", "DIY"),
    ("La vie en rose", 3, "123-4-56-789098-7", "French", "Novels"),
    ("The butler with the knife in the living room", 4, "135-7-99-143563-2", "English", "Crime fiction"),
    ("Agatha Christie investigates", 4, "423-7-13-888798-3", "English", "Crime fiction"),
    ("Sherlock Holmes is for suckers", 4, "144-1-44-144144-1", "English", "Crime fiction"),
    ("And so is Inspector Gadget", 4, "222-1-33-555555-6", "English", "Crime fiction"),
    ("All Sherlock Holmes (whom is NOT for suckers!)", 5, "000-1-22-333333-4", "English", "Crime fiction"),
    ("Il cavaliere inesistente", 6, "321-6-54-098765-4", "Italian", "Fiction"),
    ("Le cosmicomiche", 6, "111-0-66-132453-8", "Italian", "Fiction"),
    ("Vanhemmani luulivat olevansa tosi hauskoja", 7, "000-6-33-444555-5", "Finnish", "Drama"),
    ("14 veljestä - Kun inflaatio on 100%", 7, "135-0-00-111888-9", "Finnish", "Economics"),
    ("Detta är en bok", 9, "564-7-12-787878-4", "Swedish", "Truisms"),
    ("Deutsch für fast alle", 10, "132-0-73-978143-4", "German", "Languages")    

   sqlite:///chinook.db
 * sqlite:///library.db
17 rows affected.


[]

### Create the customer table

In [31]:
%%sql
CREATE TABLE customer(
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,    
    last_name TEXT NOT NULL,
    first_name TEXT NOT NULL,
    title TEXT,
    date_of_birth DATE NOT NULL,
    card_number INTEGER NOT NULL,
    address TEXT NOT NULL,
    phone CHAR(20),
    email CHAR(50)
)

   sqlite:///chinook.db
 * sqlite:///library.db
Done.


[]

Let's add a few made-up library customers.

In [32]:
%%sql
INSERT INTO customer(last_name, first_name, title, date_of_birth, card_number, address, phone, email)
VALUES
    ("Customer", "Carl", "Mr.","1/4/1988", 123456789, "This street 5, Gotham city", "+4564588799", "carl.customer@google.com"),
    ("Patron", "Patricia", "Miss", "25/12/1996", 987654321, "That street 10, Mouseville", NULL, "pattyp@yahoo.com"),
    ("Reader", "Roy", NULL, "13/2/1912", 123459876, "Rue de les livres 34, Paris", "+9178938483", NULL),
    ("Lettrice", "Lina", "Mrs.", "20/11/1967", 999999999, "Via col Vento 3, Rome", "+3904156889743", "lina-lett@altavista.it"),
    ("Book", "Bob", "Mr.", "4/7/1954", 121245456, "Leaflet street 8, Booksville", NULL, "bob.book@provider.co.uk")

   sqlite:///chinook.db
 * sqlite:///library.db
5 rows affected.


[]

### Create the loan table

In [33]:
%%sql
CREATE TABLE loan(
    loan_id INTEGER PRIMARY KEY AUTOINCREMENT,
    book_id INTEGER NOT NULL,
    customer_id INTEGER NOT NULL,
    due_date DATE NOT NULL,
    status TEXT NOT NULL,
    FOREIGN KEY (book_id)
        REFERENCES book (book_id),
    FOREIGN KEY (customer_id)
        REFERENCES customer (customer_id))

   sqlite:///chinook.db
 * sqlite:///library.db
Done.


[]

Now, let's make our customers loan a few books.

In [34]:
%%sql
INSERT INTO loan(book_id, customer_id, due_date, status)
VALUES
    (5, 3, "1.4.2021", "Returned"),
    (6, 4, "12.3.2022", "Overdue"),
    (17, 1, "15.05.2022", "Out"),
    (9, 4, "14.3.2022", "Overdue"),
    (15, 2, "5.4.2022", "Out"),
    (2, 3, "1.4.2021", "Returned"),
    (10, 4, "14.3.2022", "Overdue")

   sqlite:///chinook.db
 * sqlite:///library.db
7 rows affected.


[]

### Adding views

Below are a few views that may be useful to have if this database was for a real library. The `loans` view shows data about all loands on record; the `customer_loan_stats` view visualises how many books each customer has borrowed from the library, how many have\haven't been returned and how many loans are overdue. Finally, the `author_stats` view displays how many books there are in the library for each author, and how many books of each author have been loaned to customers.

In [35]:
%%sql
CREATE VIEW loans(loan_id, book_title, due_date, status, borrowed_by)
AS
    SELECT l.loan_id,           
           b.title, 
           l.due_date,
           l.status,
           c.last_name || ", " || c.first_name
      FROM loan l
       LEFT JOIN book b
         ON l.book_id = b.book_id
           LEFT JOIN customer c
             ON l.customer_id = c.customer_id;
            
CREATE VIEW customer_loan_stats
AS
    SELECT c.last_name || ", " || c.first_name AS customer_name,
           COUNT(l.loan_id) AS total_loans,
           COUNT(l.status) FILTER (WHERE l.status = "Out") AS active_loans,
           COUNT(l.status) FILTER (WHERE l.status = "Returned") AS returned_loans,
           COUNT(l.status) FILTER (WHERE l.status = "Overdue") AS overdue_loans
      FROM customer c
        LEFT JOIN loan l
          ON c.customer_id = l.customer_id
     GROUP BY customer_name;
    
CREATE VIEW author_stats
AS
    SELECT a.last_name || ", " || a.first_name AS author_name,
           COUNT(b.book_id) AS total_books,
           COUNT(l.loan_id) AS loaned
      FROM author a
        LEFT JOIN book b
          ON a.author_id = b.author_id
            LEFT JOIN loan l
              ON b.book_id = l.book_id
     GROUP BY author_name;    

   sqlite:///chinook.db
 * sqlite:///library.db
Done.
Done.
Done.


[]

To conclude this demo, here are the three views above in action.
`loans`

In [36]:
%%sql
SELECT * FROM loans
ORDER BY status ASC, due_date ASC

   sqlite:///chinook.db
 * sqlite:///library.db
Done.


loan_id,book_title,due_date,status,borrowed_by
3,Deutsch für fast alle,15.05.2022,Out,"Customer, Carl"
5,14 veljestä - Kun inflaatio on 100%,5.4.2022,Out,"Patron, Patricia"
2,La vie en rose,12.3.2022,Overdue,"Lettrice, Lina"
4,Sherlock Holmes is for suckers,14.3.2022,Overdue,"Lettrice, Lina"
7,And so is Inspector Gadget,14.3.2022,Overdue,"Lettrice, Lina"
1,Cómo crear títulos de libros ingeniosos,1.4.2021,Returned,"Reader, Roy"
6,How NOT to do things,1.4.2021,Returned,"Reader, Roy"


`customer_loan_stats`

In [37]:
%%sql
SELECT * FROM customer_loan_stats
ORDER BY total_loans DESC, overdue_loans DESC

   sqlite:///chinook.db
 * sqlite:///library.db
Done.


customer_name,total_loans,active_loans,returned_loans,overdue_loans
"Lettrice, Lina",3,0,0,3
"Reader, Roy",2,0,2,0
"Customer, Carl",1,1,0,0
"Patron, Patricia",1,1,0,0
"Book, Bob",0,0,0,0


`author_stats`

In [38]:
%%sql
SELECT * FROM author_stats
ORDER BY total_books DESC, loaned DESC

   sqlite:///chinook.db
 * sqlite:///library.db
Done.


author_name,total_books,loaned
"Fletcher, Jessica",4,2
"Garcia, Elena",3,1
"Black, Jack",2,1
"Pekkanen, Pekka",2,1
"Calvino, Italo",2,0
"Greve, Verena",1,1
"Perrier, Milo",1,1
"Conan Doyle, Arthur",1,0
"Karamazova, Irina",1,0
"Lindstedt, Maria",0,0
